- Tìm hiểu về các công cụ Web scraping như:
  + Beautiful soup: Khi áp dụng thì gặp nhiều vấn đề với trang Web mặc dù tốc độ nhanh nhưng lại không thu được thông tin gì
  + Selenium: Khi crawl data thì tốn quá nhiều tài nguyên cũng như thời gian chạy( 20p/ 1 page dữ liệu)
  + Scrapy: Tốc độ nhanh có thể coi là tools tốt nhất trong các loại và phù hợp với mục đích của nhóm là crawl toàn bộ dữ liệu trang web thông qua các Xpath
- Các Vấn đề gặp phải khi crawling data:
  + Thời gian thu thập dữ liệu khá mất nhiều thời gian => giải quyết bằng thư viện scrapy
  + Độ chính xác khi cào dữ liệu:
    * Thu thập chưa đúng với dữ liệu mà nhóm yêu cầu => giải quyết thay đổi các xpath chuẩn hơn dẫn tới dữ liệu cần thu thập đúng hơn
    * Thu thập chưa đủ dữ liệu => giải quyết bằng cách là tăng độ trễ giữa các yêu cầu và thêm số lần thất bại sau mỗi lần thu thập

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.http import Request

class RealEstateSpider(scrapy.Spider):
    
    name = "realestate"
    allowed_domains = ["batdongsan.vn"]
    start_urls = [f"https://batdongsan.vn/ban-can-ho-chung-cu/p{i}" for i in range(1, 215)]

    custom_settings = {
        'FEED_FORMAT': 'json',
        'FEED_URI': 'real_estate_data2.json',
        'FEED_EXPORT_ENCODING': 'utf-8',
        'DOWNLOAD_DELAY': 2,  # Thêm độ trễ giữa các yêu cầu
        'RETRY_TIMES': 3,  # Thêm số lần thử lại khi thất bại
        'LOG_LEVEL': 'INFO',  # Đặt mức log để theo dõi tiến trình
    }

    def parse(self, response):
        blocks = response.xpath("//div[contains(@class, 'uk-grid uk-grid-small uk-grid-width-1-1')]//div[contains(@class, 'item')]")
        for block in blocks:
            link = block.xpath(".//div[@class='name']/a/@href").get()
            if link:
                link = response.urljoin(link)
                yield scrapy.Request(url=link, callback=self.parse_details, errback=self.errback_httpbin, dont_filter=True)

    def parse_details(self, response):
        price = response.xpath("//strong[contains(@class, 'price')]/text()").get()
        if not price:
            price = response.xpath("//strong[@class='price']/text()").get()

        params = response.xpath("//ul[contains(@class, 'uk-list')]//li")
        parameters = []
        for param in params:
            name = param.xpath("normalize-space(.//strong/text())").get()
            value = param.xpath("normalize-space(string())").get()
            if name and value:
                parameters.append(f"{name}: {value.replace(name, '').strip()}")

        # Extracting the specific "Nội dung tin đăng" content
        details_section = response.xpath("//h3[contains(@class, 'uk-panel-title') and span/text()='Nội dung tin đăng']/following-sibling::div")
        details = details_section.xpath(".//text()").getall()
        details = " ".join(detail.strip() for detail in details if detail.strip())

        yield {
            'Link': response.url,
            'Price': price.strip() if price else None,
            'Parameters': "\n".join(parameters),
            'Details': details,
        }

    def errback_httpbin(self, failure):
        # log all failures
        self.logger.error(repr(failure))

        if failure.check(HttpError):
            # these are HTTP errors
            response = failure.value.response
            self.logger.error('HttpError on %s', response.url)

        elif failure.check(DNSLookupError):
            # this is the original request
            request = failure.request
            self.logger.error('DNSLookupError on %s', request.url)

        elif failure.check(TimeoutError, TCPTimedOutError):
            request = failure.request
            self.logger.error('TimeoutError on %s', request.url)

# Run the Spider
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',
    'FEED_URI': 'real_estate_data2.json',
    'FEED_EXPORT_ENCODING': 'utf-8',
    'DOWNLOAD_DELAY': 2,  # Thêm độ trễ giữa các yêu cầu
    'RETRY_TIMES': 3,  # Thêm số lần thử lại khi thất bại
    'LOG_ENABLED': True
})

process.crawl(RealEstateSpider)
process.start()


2024-06-06 21:13:54 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-06-06 21:13:54 [scrapy.utils.log] INFO: Versions: lxml 5.2.2.0, libxml2 2.12.6, cssselect 1.2.0, parsel 1.9.1, w3lib 2.1.2, Twisted 24.3.0, Python 3.12.0 (v3.12.0:0fb18b02c8, Oct  2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 24.1.0 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform macOS-14.4.1-arm64-arm-64bit
2024-06-06 21:13:54 [scrapy.addons] INFO: Enabled addons:
[]
2024-06-06 21:13:54 [py.warnings] WARNING: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future ver

- Cấu trúc và chức năng:
  + RealEstateSpider là lớp spider chính, bắt đầu thu thập dữ liệu từ danh sách các trang chứa danh sách các căn hộ chung cư.
  + parse là hàm đầu tiên xử lý từng trang, tìm kiếm các link chi tiết của từng căn hộ và gọi hàm parse_details để xử lý chi tiết từng căn hộ.
  + parse_details là hàm xử lý các thông tin chi tiết của từng căn hộ như giá, các thông số và nội dung tin đăng.
- Cấu hình:
  + custom_settings được sử dụng để thiết lập các cấu hình cụ thể cho spider này như định dạng và đường dẫn file xuất, mã hóa, độ trễ tải xuống giữa các yêu cầu và số lần thử lại khi gặp lỗi.
  + Các cấu hình này giúp tối ưu hóa hiệu suất và độ tin cậy của spider.
- Xử lý lỗi:
  + Hàm errback_httpbin được sử dụng để xử lý các lỗi xảy ra trong quá trình gửi yêu cầu HTTP, bao gồm các lỗi như HttpError, DNSLookupError, và TimeoutError.
  + Điều này giúp đảm bảo rằng spider có thể tiếp tục chạy và ghi nhận lại các lỗi để có thể xử lý sau này.
- Xuất dữ liệu:
  + Dữ liệu thu thập được xuất ra file JSON với tên là real_estate_data2.json và được mã hóa UTF-8 để đảm bảo tính toàn vẹn của dữ liệu khi chứa các ký tự đặc biệt.
- Log và giám sát:
  + Mức độ log được đặt là INFO để giúp giám sát tiến trình của spider mà không gây quá tải với quá nhiều thông tin chi tiết.
- Hiệu quả và tin cậy:
  + Đoạn mã đã thêm độ trễ giữa các yêu cầu và thiết lập số lần thử lại để tăng cường độ tin cậy khi gặp các lỗi mạng hoặc server.
Việc sử dụng dont_filter=True trong yêu cầu chi tiết giúp đảm bảo rằng tất cả các link được xử lý mà không bị bỏ qua do bộ lọc URL của Scrapy.

- Ưu điểm
  + Độ trễ giữa các yêu cầu: Giúp giảm tải cho máy chủ, tránh bị chặn.
  + Thử lại khi thất bại: Tăng khả năng hoàn thành thu thập dữ liệu ngay cả khi gặp lỗi.
  + Ghi log chi tiết: Giúp dễ dàng theo dõi tiến trình và xử lý sự cố.
- Nhược điểm
  + Tốc độ thu thập chậm: Do có độ trễ giữa các yêu cầu, tốc độ thu thập dữ liệu sẽ chậm hơn.
  + Xử lý lỗi chưa toàn diện: Một số lỗi không được xử lý cụ thể có thể gây gián đoạn tiến trình.
  + Phụ thuộc vào cấu trúc trang web: Nếu cấu trúc trang web thay đổi, mã sẽ không hoạt động chính xác và cần phải cập nhật lại.